![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [20]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/Users/dodkins/PythonProjects/stocks/options_testing/options_testing')

import os
from datetime import datetime 
import pandas as pd
import opstrat as op

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
from qc_simulator import QuantBook, Resolution, OptionRight

In [22]:
QuantBook()

In [23]:
import quantconnect
from utils import aggregate, concat_dfs, get_start_price
from technical_analysis import get_poc
# from strategies import measure_period_profit
# from strategies import add_expirations

In [24]:
qbw = quantconnect.QuantBookWrapper({'qb':QuantBook(),'Resolution':Resolution,'OptionRight':OptionRight})

In [25]:
qb = QuantBook()
tsla = qb.AddEquity("TSLA")
equity_symbol = tsla.Symbol

In [26]:
qb.History(qb.Securities.Keys, 3000, Resolution.Hour)

open        high         low       close  \
symbol time                                                                  
lol    2020-12-30 10:00:00  224.026644  225.026644  222.799978  224.404978   
       2020-12-30 11:00:00  224.409978  225.669977  223.693311  224.606644   
       2020-12-30 12:00:00  224.619978  227.833311  224.596644  227.166644   
       2020-12-30 13:00:00  227.136644  228.999977  226.999977  228.066644   
       2020-12-30 14:00:00  228.033311  229.816644  227.666644  229.376644   
...                                ...         ...         ...         ...   
       2022-09-13 12:00:00  293.680000  293.990000  290.400000  291.000000   
       2022-09-13 13:00:00  291.000000  293.280000  290.670000  292.300000   
       2022-09-13 14:00:00  292.270000  295.500000  291.780000  294.830000   
       2022-09-13 15:00:00  294.830000  295.970000  293.000000  294.100000   
       2022-09-13 16:00:00  294.120000  294.990000  291.400000  292.130000   

                                volume  
symbol time                             
lol    2020-12-30 10:00:00  12666775.0  
       2020-12-30 11:00:00  16660004.0  
       2020-12-30 12:00:00  21427847.0  
       2020-12-30 13:00:00  15279809.0  
       2020-12-30 14:00:00  12887422.0  
...                                ...  
       2022-09-13 12:00:00   7883773.0  
       2022-09-13 13:00:00   5298142.0  
       2022-09-13 14:00:00   9703434.0  
       2022-09-13 15:00:00   7136008.0  
       2022-09-13 16:00:00  10745308.0  

[3000 rows x 5 columns]

In [27]:
from strategies import IronCondors, measure_period_profit
import plots

In [28]:
ic_strat = IronCondors(long_offset=15, short_offset=15, qbw=qbw)

In [29]:
tsla = qbw.get_tsla()

In [30]:
tsla

,open,high,low,close,volume
time,,,,,
2022-08-03 13:00:00,304.746636,309.109969,303.333303,307.993303,10662028.0
2022-08-03 14:00:00,307.943303,308.416636,306.036636,307.446636,6186187.0
2022-08-03 15:00:00,307.473303,309.466636,306.823303,307.983303,7723828.0
2022-08-03 16:00:00,307.993303,308.756636,307.296636,307.396636,10598359.0
2022-08-04 10:00:00,311.236636,313.626635,308.866636,311.539969,14604838.0
...,...,...,...,...,...
2022-09-13 12:00:00,293.680000,293.990000,290.400000,291.000000,7883773.0
2022-09-13 13:00:00,291.000000,293.280000,290.670000,292.300000,5298142.0
2022-09-13 14:00:00,292.270000,295.500000,291.780000,294.830000,9703434.0


In [31]:
ic_strat = IronCondors(long_offset=15, short_offset=15, qbw=qbw)
ic = measure_period_profit(tsla,  
    ic_strat,
    expiration='week',
    update_freq='once')
plots.plot_candles_and_profit(ic, lines=ic_strat.legs)

In [32]:
plots.plot_candles_and_profit(ic, lines=ic_strat.legs)

In [33]:
concise_cols = ['date', 'underlying_open', 'underlying_close', 'hourly_profit', 'running_profit', 'new_option', 'start_price', 'sell_call_strike', 'buy_call_strike', 'sell_put_strike', 'buy_put_strike', 'sell_call_strike_open', 'sell_call_strike_close', 'buy_call_strike_open', 'buy_call_strike_close', 'sell_put_strike_open', 'sell_put_strike_close', 'buy_put_strike_open', 'buy_put_strike_close']
ic[concise_cols]

,date,underlying_open,underlying_close,hourly_profit,running_profit,new_option,start_price,sell_call_strike,buy_call_strike,sell_put_strike,buy_put_strike,sell_call_strike_open,sell_call_strike_close,buy_call_strike_open,buy_call_strike_close,sell_put_strike_open,sell_put_strike_close,buy_put_strike_open,buy_put_strike_close
0,2022-08-03 13:00:00,304.746636,307.993303,-0.002100,-0.002100,True,304.746636,351.0,357.0,261.0,255.0,0.002235,0.005456,-0.000489,-0.001299,5.476585e-04,2.003592e-04,-5.485496e-05,-1.781226e-05
1,2022-08-03 14:00:00,307.943303,307.446636,0.000872,-0.001228,False,304.746636,351.0,357.0,261.0,255.0,0.004870,0.004257,-0.001132,-0.000977,1.755880e-04,2.058429e-04,-1.505614e-05,-1.798100e-05
2,2022-08-03 15:00:00,307.473303,307.983303,-0.000098,-0.001327,False,304.746636,351.0,357.0,261.0,255.0,0.003859,0.004440,-0.000866,-0.001010,1.756369e-04,1.487841e-04,-1.474151e-05,-1.224767e-05
3,2022-08-03 16:00:00,307.993303,307.396636,0.000778,-0.000549,False,304.746636,351.0,357.0,261.0,255.0,0.004002,0.003387,-0.000890,-0.000741,1.266361e-04,1.542405e-04,-1.000481e-05,-1.247093e-05
4,2022-08-04 10:00:00,311.236636,311.539969,0.001966,0.001417,False,304.746636,351.0,357.0,261.0,255.0,0.000853,0.000952,-0.000115,-0.000130,4.137666e-07,3.558719e-07,-9.047890e-09,-7.646148e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2022-09-13 12:00:00,293.680000,291.000000,-0.010047,0.222159,False,300.590000,347.0,353.0,257.0,251.0,0.002967,0.001510,-0.000835,-0.000403,2.250231e-02,3.864908e-02,-4.816273e-03,-8.868738e-03
196,2022-09-13 13:00:00,291.000000,292.300000,0.007353,0.229512,False,300.590000,347.0,353.0,257.0,251.0,0.001387,0.001939,-0.000365,-0.000524,3.679018e-02,2.830140e-02,-8.316725e-03,-6.182659e-03
197,2022-09-13 14:00:00,292.270000,294.830000,0.008495,0.238007,False,300.590000,347.0,353.0,257.0,251.0,0.001770,0.003370,-0.000471,-0.000944,2.700553e-02,1.578539e-02,-5.811852e-03,-3.172854e-03
198,2022-09-13 15:00:00,294.830000,294.100000,-0.000739,0.237268,False,300.590000,347.0,353.0,257.0,251.0,0.003116,0.002597,-0.000860,-0.000706,1.486910e-02,1.739364e-02,-2.938121e-03,-3.505510e-03


In [58]:
ic_strat = IronCondors(long_offset=10, short_offset=10, qbw=qbw)
ic = measure_period_profit(tsla,  
    ic_strat,
    expiration='week',
    update_freq='once')
plots.plot_candles_and_profit(ic, lines=ic_strat.legs)

strikes [213. 219. 225. 231. 237. 243. 249. 255. 261. 267. 273. 279. 285. 291.
 297. 303. 309. 315. 321. 327. 333. 339. 345. 351. 357. 363. 369. 375.
 381. 387. 393.] 333.0
strikes [213. 219. 225. 231. 237. 243. 249. 255. 261. 267. 273. 279. 285. 291.
 297. 303. 309. 315. 321. 327. 333. 339. 345. 351. 357. 363. 369. 375.
 381. 387. 393.] 339.0
strikes [213. 219. 225. 231. 237. 243. 249. 255. 261. 267. 273. 279. 285. 291.
 297. 303. 309. 315. 321. 327. 333. 339. 345. 351. 357. 363. 369. 375.
 381. 387. 393.] 273.0
strikes [213. 219. 225. 231. 237. 243. 249. 255. 261. 267. 273. 279. 285. 291.
 297. 303. 309. 315. 321. 327. 333. 339. 345. 351. 357. 363. 369. 375.
 381. 387. 393.] 273.0
buy leg 267.0
strikes [202.         207.66666667 213.33333333 219.         224.66666667
 230.33333333 236.         241.66666667 247.33333333 253.
 258.66666667 264.33333333 270.         275.66666667 281.33333333
 287.         292.66666667 298.33333333 304.         309.66666667
 315.33333333 321.         326

In [36]:
strikes = [247.5, 150. , 192.5, 380. , 227.5 ,160. , 410.,  210.,  145.,  390.,  355.,  370.,
 300. , 265.,  165.,  275.,  110.,  180.,  385.,  280.,  340.,  325.,  255.,  430.,
 100. , 395.,  140.,  215.,  212.5, 260.,  295.,  480.,  240.,  217.5, 175.,  185.,
 235. , 305.,  440.,  460.,  450.,  230.,  310.,  120.,  222.5, 250.,  202.5, 335.,
 200. , 225.,  170.,  237.5, 470.,  285.,  315.,  345.,  245.,  130.,  330.,  420.,
 400. , 375.,  187.5, 195.,  232.5, 207.5, 205.,  290.,  350.,  320.,  360.,  190.,
 197.5, 365.,  252.5, 242.5, 155.,  270.,  220., ]

In [37]:
sorted_strikes = np.sort(strikes)

In [55]:
sorted_strikes - 240.

array([-140. , -130. , -120. , -110. , -100. ,  -95. ,  -90. ,  -85. ,
        -80. ,  -75. ,  -70. ,  -65. ,  -60. ,  -55. ,  -52.5,  -50. ,
        -47.5,  -45. ,  -42.5,  -40. ,  -37.5,  -35. ,  -32.5,  -30. ,
        -27.5,  -25. ,  -22.5,  -20. ,  -17.5,  -15. ,  -12.5,  -10. ,
         -7.5,   -5. ,   -2.5,    0. ,    2.5,    5. ,    7.5,   10. ,
         12.5,   15. ,   20. ,   25. ,   30. ,   35. ,   40. ,   45. ,
         50. ,   55. ,   60. ,   65. ,   70. ,   75. ,   80. ,   85. ,
         90. ,   95. ,  100. ,  105. ,  110. ,  115. ,  120. ,  125. ,
        130. ,  135. ,  140. ,  145. ,  150. ,  155. ,  160. ,  170. ,
        180. ,  190. ,  200. ,  210. ,  220. ,  230. ,  240. ])

In [52]:
closest_inds = np.argpartition(np.abs(sorted_strikes - 240.), 5)[:5]
closest_inds

array([33, 36, 35, 34, 37])

In [43]:
sorted_strikes[35], sorted_strikes[36]

(240.0, 242.5)

In [53]:
sorted_strikes[closest_inds]

array([235. , 242.5, 240. , 237.5, 245. ])

In [56]:
sorted_strikes[sorted_strikes > 240].min() 

242.5

In [57]:
sorted_strikes[sorted_strikes < 240.].max()

237.5